# Run logs

cycled once of 30 epochs
lrmain = .000001
lrlast = .0001
optim1 = optim.Adam(
```
    [
        {"params":model_1.resnet_model.parameters(),"lr": lrmain},
        {"params":model_1.x1.parameters(), "lr": lrlast},
        {"params":model_1.x1a.parameters(), "lr": lrlast},
        {"params":model_1.x2.parameters(), "lr": lrlast},
        {"params":model_1.x2a.parameters(), "lr": lrlast},
       {"params":model_1.x3.parameters(), "lr": lrlast},
       {"params":model_1.x3a.parameters(), "lr": lrlast},
       {"params":model_1.y1o.parameters(), "lr": lrlast},
         {"params":model_1.y2o.parameters(), "lr": lrlast},
       
       
   ])
```

## Epoch 25/29

train total loss: 1.6803 dnd_race loss: 0.7489 dnd_class loss: 0.9314
train race_acc: 0.8049 class_acc: 0.6280
val total loss: 1.6511 dnd_race loss: 0.7321 dnd_class loss: 0.9190
val race_acc: 0.8250 class_acc: 0.6375
saving with loss of 1.6511068105697633 improved over previous 1.659845781326294

this score is better than the single model which scored 80% on the race task on its own. 


# Run 2

## Epoch 8/29

train total loss: 1.6213 dnd_race loss: 0.7504 dnd_class loss: 0.8709
train race_acc: 0.7957 class_acc: 0.6829
val total loss: 1.5756 dnd_race loss: 0.7225 dnd_class loss: 0.8531
val race_acc: 0.8500 class_acc: 0.6750
saving with loss of 1.5756343841552733 improved over previous 1.5936285972595214


## Epoch 10/29

train total loss: 1.6042 dnd_race loss: 0.7130 dnd_class loss: 0.8912
train race_acc: 0.8415 class_acc: 0.6433
val total loss: 1.6003 dnd_race loss: 0.7496 dnd_class loss: 0.8506
val race_acc: 0.8250 class_acc: 0.7125

both of these epochs were good

# Run 3

Adjusted learning rate to be slightly higher. helped get above on both tasks
```
lrmain = .00001
lrlast = .0001
optim1 = optim.Adam(
    [
        {"params":model_1.resnet_model.parameters(),"lr": lrmain},
        {"params":model_1.x1.parameters(), "lr": lrlast},
        {"params":model_1.x1a.parameters(), "lr": lrlast},
        {"params":model_1.x2.parameters(), "lr": lrlast},
        {"params":model_1.x2a.parameters(), "lr": lrlast},
       {"params":model_1.x3.parameters(), "lr": lrlast},
       {"params":model_1.x3a.parameters(), "lr": lrlast},
       {"params":model_1.y1o.parameters(), "lr": lrlast},
         {"params":model_1.y2o.parameters(), "lr": lrlast},
       
       
   ])

#optim1 = optim.Adam(model_1.parameters(), lr=0.01)#,momentum=.9)
# Observe that all parameters are being optimized
optimizer_ft = optim1

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=4, gamma=0.1)

```

Epoch 6/29
----------
train total loss: 1.5900 dnd_race loss: 0.7520 dnd_class loss: 0.8380
train race_acc: 0.7927 class_acc: 0.7195
val total loss: 1.5751 dnd_race loss: 0.7474 dnd_class loss: 0.8277
val race_acc: 0.8375 class_acc: 0.7500
saving with loss of 1.5751248002052307 improved over previous 1.5785507321357728


In [1]:
# License: BSD
# Author: Sasank Chilamkurthy

from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from random import randrange
import torch.nn.functional as F


plt.ion()   # interactive mode

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split


dat_race = pd.read_csv('dnd_race.csv')
dat_class = pd.read_csv('dnd_class.csv')

dat_race['image_path'] = dat_race['label']+'/'+dat_race['image_number']
dat_class['image_path'] = dat_class['label']+'/'+dat_class['image_number']

dat_race = dat_race.rename(index=str, columns={"label": "race"})
dat_class = dat_class.rename(index=str, columns={"label": "class"})
dat_class.head(1)


X_race = dat_race['image_path']
X_class = dat_class['image_path']

y_race = pd.get_dummies(dat_race['race'])
y_class = pd.get_dummies(dat_class['class'])


In [3]:
dat_class['class'].value_counts()

rogue      148
paladin    147
wizard     119
Name: class, dtype: int64

In [4]:
148/dat_class.shape[0]

0.357487922705314

In [5]:
dat_race.race.value_counts()

elf           188
gnome         122
dragonborn    120
Name: race, dtype: int64

In [6]:
188/dat_race.shape[0]

0.4372093023255814

In [7]:
X_race_train, X_race_test, y_race_train, y_race_test = train_test_split(X_race, y_race, test_size=0.20, random_state=42)
X_class_train, X_class_test, y_class_train, y_class_test = train_test_split(X_class, y_class, test_size=0.20, random_state=42)


In [8]:
X_race_train = X_race_train.values.tolist()
X_race_test = X_race_test.values.tolist()


X_class_train = X_class_train.values.tolist()
X_class_test = X_class_test.values.tolist()

race_shape = y_race_train.shape[1]
class_shape = y_class_train.shape[1]


y_race_train = y_race_train.values.tolist()
y_race_test = y_race_test.values.tolist()

y_class_train = y_class_train.values.tolist()
y_class_test = y_class_test.values.tolist()

In [9]:
len(y_race_train)

344

In [10]:
len(y_class_train)

331

In [11]:
#for i in dataset_style:
#    print(i,dat[i].sum(),dat[i].sum()/dat.shape[0])

In [12]:
#for i in dataset_rest:
#    print(i,dat[i].sum(),dat[i].sum()/dat.shape[0])

In [13]:
import random
flip = random.randint(0, 1)
flip

0

In [14]:
class fgo_dataset(Dataset):
    def __init__(self,king_of_lists, transform=None):
        """
        gender_img, style_img, gender_pred,style_pred 
        
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.king_of_lists = king_of_lists
        self.transform = transform
        
    def __getitem__(self,index):
        
        #random_index = randrange(len(self.king_of_lists[index]))
        
        img1 = Image.open('images/'+self.king_of_lists[0][index])
        img1 = img1.convert('RGB')
        
        #img2 = Image.open('images/'+self.king_of_lists[1][index])
        #img2 = img2.convert('RGB')
        
        
        gender = self.king_of_lists[1][index] # gender
        #color = self.king_of_lists[3][index] # region
        
        if self.transform is not None:
            img1 = self.transform(img1)

        list_of_labels = [torch.from_numpy(np.array(gender))]
       
        return img1, list_of_labels[0]
    
    def __len__(self):
        return len(self.king_of_lists[0])

In [15]:
batch_size = 8
# Data augmentation and normalization for training
# Just normalization for validation
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((256,256)),
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize((224,224)),
        #transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

train_lists = [X_race_train, y_race_train]
test_lists = [X_race_test, y_race_test ]

training_dataset = fgo_dataset(king_of_lists = train_lists,
                               transform = data_transforms['train'] )

test_dataset = fgo_dataset(king_of_lists = test_lists,
                           transform = data_transforms['val'] )

dataloaders_dict = {'train': torch.utils.data.DataLoader(training_dataset, batch_size=batch_size, shuffle=True, num_workers=0),
                   'val':torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=True, num_workers=0)
                   }
dataset_sizes = {'train':len(train_lists[0]),
                'val':len(test_lists[0])}


train_class_lists = [X_class_train, y_class_train]
test_class_lists = [X_class_test, y_class_test ]

training_style_dataset = fgo_dataset(king_of_lists = train_class_lists,
                               transform = data_transforms['train'] )

test_style_dataset = fgo_dataset(king_of_lists = test_class_lists,
                           transform = data_transforms['val'] )

dataloaders_style_dict = {'train': torch.utils.data.DataLoader(training_style_dataset, batch_size=batch_size,drop_last =True, shuffle=True, num_workers=0),
                   'val':torch.utils.data.DataLoader(test_style_dataset, batch_size=batch_size,drop_last =True, shuffle=True, num_workers=0)
                   }
dataloaders_style_sizes = {'train':len(train_lists[0]),
                'val':len(test_lists[0])}

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [16]:
print(len(dataloaders_style_dict['train']),len(dataloaders_style_dict['val']))

41 10


In [17]:
print(len(dataloaders_dict['train']),len(dataloaders_dict['val']))

43 11


In [18]:
model_ft = models.resnet50(pretrained=False)
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, 3)
#for param in model_ft.parameters():
#    param.requires_grad = False

#model_ft.load_state_dict(torch.load('./models/dnd_resnet_50.pth'))

print(model_ft)
#num_ftrs = model_ft.classifier[6].in_features
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, 512)

#race_shape = y_race_train.shape[1]
#class_shape = y_class_train.shape[1]
class multi_output_model(torch.nn.Module):
    def __init__(self, model_core):
        super(multi_output_model, self).__init__()
        
        self.resnet_model = model_core
        
        self.x1 =  nn.Linear(512,512)
        nn.init.xavier_normal_(self.x1.weight)
        self.x1a =  nn.Linear(512,512)
        nn.init.xavier_normal_(self.x1a.weight)
        
        self.x2 =  nn.Linear(512,256)
        nn.init.xavier_normal_(self.x2.weight)
        self.x2a =  nn.Linear(256,256)
        nn.init.xavier_normal_(self.x2a.weight)
        
        self.x3 =  nn.Linear(256,128)
        nn.init.xavier_normal_(self.x3.weight)
        self.x3a =  nn.Linear(128,128)
        nn.init.xavier_normal_(self.x3a.weight)
        #comp head 1
        
        
        #heads
        self.y1o = nn.Linear(128,race_shape)
        nn.init.xavier_normal_(self.y1o.weight)#
        self.y2o = nn.Linear(128,class_shape)
        nn.init.xavier_normal_(self.y2o.weight)
        
    def forward(self, x):
       
        x1 = self.resnet_model(x)
        #x1 =  F.relu(self.x1(x1))
        #x1 =  F.relu(self.x2(x1))
        
        x1 =  F.relu(self.x1(x1))
        x1 =  F.relu(self.x1a(x1))
        x1 =  F.relu(self.x2(x1))
        x1 =  F.relu(self.x2a(x1))
        
        x1 =  F.relu(self.x3(x1))
        x1 =  F.relu(self.x3a(x1))
        
        
        #x = F.relu(self.x2(x))
        #x1 = F.relu(self.x3(x))
        
        # heads
        y1o = F.softmax(self.y1o(x1),dim=1)
        y2o = F.softmax(self.y2o(x1),dim=1)
        #y2o = torch.sigmoid(self.y2o(x1)) #should be sigmoid


        return y1o, y2o
model_1 = multi_output_model(model_ft)
model_1 = model_1.to(device)
model_1.load_state_dict(torch.load('./models/masked_resnet34.pth'))

print(model_1)
print(model_1.parameters())    
criterion = [nn.CrossEntropyLoss(),nn.CrossEntropyLoss()]

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=F

)
multi_output_model(
  (resnet_model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace)
        (downsample): Sequential(
  

)
<generator object Module.parameters at 0x0000014F2E630930>


In [19]:
it = iter(dataloaders_style_dict['val'])
gender_input ,gender = next(it)
x1,x2 = model_1(gender_input.to(device) )
x1,x2


(tensor([[0.0061, 0.9781, 0.0158],
         [0.0085, 0.9873, 0.0041],
         [0.0045, 0.9476, 0.0479],
         [0.0084, 0.1048, 0.8868],
         [0.2636, 0.0364, 0.7000],
         [0.2478, 0.5930, 0.1591],
         [0.0108, 0.1503, 0.8389],
         [0.9203, 0.0443, 0.0354]], device='cuda:0', grad_fn=<SoftmaxBackward>),
 tensor([[0.0908, 0.7815, 0.1277],
         [0.0354, 0.0249, 0.9397],
         [0.0303, 0.9148, 0.0549],
         [0.1747, 0.7454, 0.0799],
         [0.6794, 0.2739, 0.0466],
         [0.4274, 0.3432, 0.2294],
         [0.0887, 0.8812, 0.0300],
         [0.7992, 0.0723, 0.1285]], device='cuda:0', grad_fn=<SoftmaxBackward>))

In [20]:
from torch.autograd import Variable
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()
    print('starting')
    best_model_wts = copy.deepcopy(model.state_dict())
    best_loss = 100

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                scheduler.step()
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_loss0= 0.0
            running_loss1= 0.0
            
            #color_corrects= []
            #total_colors = []
            gender_corrects = 0.0
            color_corrects= 0.0
            
            num_class_preds = 0
            num_gender_preds = 0 
            # Iterate over data.
            gender_iterator = iter(dataloaders_dict[phase])

            style_iterator = iter(dataloaders_style_dict[phase])
            
            #number_of_batches = len(dataloaders_style_dict[phase]) + len(dataloaders_dict[phase])
            for i in range(len(dataloaders_style_dict[phase])):
                #dataloaders_dict , dataloaders_style_dict
                
                ### gender
                inputs,labels_ = next(gender_iterator)
                
                inputs = inputs.to(device) 
                labels_ = labels_.to(device)
                
                outputs = model(inputs)
                preds_ = outputs[0]
                
                loss_gender = criterion[0](preds_, torch.max(labels_.float(), 1)[1]).to(device)
                running_loss0 += loss_gender.item() * inputs.size(0)
                gender_corrects += torch.sum(torch.max(preds_, 1)[1] == torch.max(labels_, 1)[1])
                num_gender_preds += inputs.size(0)
                #loss = loss_gender
                #running_loss += loss.item() * inputs.size(0)
                
                #optimizer.zero_grad()
                
                #if phase == 'train':
                        
                #        loss.backward(retain_graph=True)
                #        optimizer.step()
                        
                #####################################################################################
                ### style
                inputs,labels_ = next(style_iterator)
                inputs = inputs.to(device) 
                labels_ = labels_.to(device)
                
                outputs = model(inputs)
                preds_ = outputs[1]

                loss_style = criterion[1](preds_, torch.max(labels_.float(), 1)[1]).to(device)
                running_loss1 += loss_style.item() * inputs.size(0)
                color_corrects += torch.sum(torch.max(preds_, 1)[1] == torch.max(labels_, 1)[1])
                num_class_preds += inputs.size(0)
                #loss = loss_style
                
                
                optimizer.zero_grad()
                
                loss = loss_style + loss_gender
                running_loss += loss.item() * inputs.size(0)*2
                if phase == 'train':
                        
                    loss.backward()
                    optimizer.step()
            
                
            epoch_loss = running_loss / (num_class_preds +num_gender_preds)
            epoch_loss0 = running_loss0 / (num_gender_preds)
            epoch_loss1 = running_loss1 / (num_class_preds)

            
            color_acc = color_corrects.double()/num_class_preds
            gender_corrects = gender_corrects.double() / num_gender_preds
            
            print('{} total loss: {:.4f} dnd_race loss: {:.4f} dnd_class loss: {:.4f}'.format(phase,epoch_loss,epoch_loss0,
                                                                                                          epoch_loss1))
            print('{} race_acc: {:.4f} '
                  'class_acc: {:.4f}'.format(
                phase, gender_corrects,color_acc))

            if phase == 'val' and epoch_loss < best_loss:
                print('saving with loss of {}'.format(epoch_loss),
                      'improved over previous {}'.format(best_loss))
                best_loss = epoch_loss
                best_model_wts = copy.deepcopy(model.state_dict())
                torch.save(best_model_wts, './models/masked_resnet34_run2.pth')


        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(float(best_loss)))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [21]:
lrmain = .00001
lrlast = .0001
optim1 = optim.Adam(
    [
        {"params":model_1.resnet_model.parameters(),"lr": lrmain},
        {"params":model_1.x1.parameters(), "lr": lrlast},
        {"params":model_1.x1a.parameters(), "lr": lrlast},
        {"params":model_1.x2.parameters(), "lr": lrlast},
        {"params":model_1.x2a.parameters(), "lr": lrlast},
       {"params":model_1.x3.parameters(), "lr": lrlast},
       {"params":model_1.x3a.parameters(), "lr": lrlast},
       {"params":model_1.y1o.parameters(), "lr": lrlast},
         {"params":model_1.y2o.parameters(), "lr": lrlast},
       
       
   ])

#optim1 = optim.Adam(model_1.parameters(), lr=0.01)#,momentum=.9)
# Observe that all parameters are being optimized
optimizer_ft = optim1

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=4, gamma=0.1)

In [22]:
model_ft1 = train_model(model_1, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=30)

starting
Epoch 0/29
----------


D:\Users\micha\Anaconda3\envs\tensorflow_env\lib\site-packages\PIL\Image.py:965: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  ' expressed in bytes should be converted ' +
D:\Users\micha\Anaconda3\envs\tensorflow_env\lib\site-packages\PIL\TiffImagePlugin.py:785: UserWarning: Corrupt EXIF data.  Expecting to read 4 bytes but only got 0. 
  warnings.warn(str(msg))


train total loss: 1.7117 dnd_race loss: 0.7663 dnd_class loss: 0.9454
train race_acc: 0.7805 class_acc: 0.6280
val total loss: 1.6661 dnd_race loss: 0.7623 dnd_class loss: 0.9038
val race_acc: 0.7750 class_acc: 0.6375
saving with loss of 1.6660812377929688 improved over previous 100

Epoch 1/29
----------
train total loss: 1.6628 dnd_race loss: 0.7617 dnd_class loss: 0.9011
train race_acc: 0.7896 class_acc: 0.6341
val total loss: 1.6164 dnd_race loss: 0.7475 dnd_class loss: 0.8688
val race_acc: 0.8375 class_acc: 0.6875
saving with loss of 1.6163779377937317 improved over previous 1.6660812377929688

Epoch 2/29
----------
train total loss: 1.6305 dnd_race loss: 0.7525 dnd_class loss: 0.8780
train race_acc: 0.7866 class_acc: 0.6463
val total loss: 1.5786 dnd_race loss: 0.7479 dnd_class loss: 0.8306
val race_acc: 0.8000 class_acc: 0.7000
saving with loss of 1.5785507321357728 improved over previous 1.6163779377937317

Epoch 3/29
----------
train total loss: 1.5721 dnd_race loss: 0.7498 dn

KeyboardInterrupt: 

# Eval
 